In [25]:
import numpy as np  
import pandas as pd
from libsvm.svmutil import * 
from sklearn.model_selection import train_test_split

In [26]:
def read_data(file_path):
    x, label = [], []
    with open(file_path, "r") as f:
        for idx, line in enumerate(f.readlines()):
            tokens = [tokens for tokens in line.split()]
            label.append(int(tokens[0]))
            f = {}
            for token in tokens[1:]:
                k, v = token.split(":")
                f[k] = float(v)
            x.append(f)
    x = pd.DataFrame(x, columns=[str(i)
                     for i in range(1, 37)]).fillna(0).to_numpy()
    y = np.array(label).reshape((-1,))
    return y, x


In [27]:
def Makedata(filename):
    y = []
    x = []
    with open(filename, "r") as file:
        f = file.readlines()
        for lines in f:
            x_in_line = [0]*36
            line = lines.split()
            y.append(int(line[0]))
            for pair in line[1:]:
                index, val = pair.split(':')
                index = int(index)
                val = float(val)
                x_in_line[index-1] = val
            x.append(x_in_line)
    y = np.array(y).reshape((-1,))
    x = np.array(x)
    return  y, x

In [28]:
dtrainy, dtrainx = svm_read_problem('satimage.scale')
dtesty, dtestx = svm_read_problem('satimage.scale.t')

In [126]:
#q11
trainy0 , testy0 = [], []
for i in range(len(dtrainy)):
    if dtrainy[i] == 5:
        trainy0.append(1)
    else: 
        trainy0.append(-1)
m = svm_train(trainy0, dtrainx, "-t 0 -c 10 -q")
SV = []
for lines in m.get_SV():
    x_in_line = [0]*36
    for i in range(36):
        if i in lines:
            x_in_line[i] = lines[i]
        else:
            x_in_line[i] = 0
    SV.append(x_in_line)
w = np.dot(np.array(SV).transpose(), np.array(m.get_sv_coef()))
dist = np.sqrt(np.square(w).sum())
print("dist of w:", dist)

dist of w: 4.623086501965068


In [127]:
# q12 13
numbers = [1, 2, 3, 4, 5]
for n in numbers:   
    trainy1 , testy1 = [], []
    for i in range(len(dtrainy)):
        if dtrainy[i] == n:
            trainy1.append(1)
        else: 
            trainy1.append(-1)
    for i in range(len(dtesty)):
        if dtesty[i] == n:
            testy1.append(1)
        else: 
            testy1.append(-1)
    m = svm_train(trainy1, dtrainx, "-t 1 -g 1 -d 3 -r 1 -c 10 -q")  
    acc = svm_predict(testy1, dtestx, m, "-q")[1][0]
    print(n, "Ein: ", round(1-acc/100, 3), "SV num:", m.l)


1 Ein:  0.009 SV num: 171
2 Ein:  0.008 SV num: 93
3 Ein:  0.048 SV num: 385
4 Ein:  0.08 SV num: 659
5 Ein:  0.035 SV num: 281


In [128]:
# q14
trainy2, testy2 = [], []
for i in range(len(dtrainy)):
    if dtrainy[i] == 1:
        trainy2.append(1)
    else:
        trainy2.append(-1)
for i in range(len(dtesty)):
    if dtesty[i] == 1:
        testy2.append(1)
    else:
        testy2.append(-1)

numbers = [0.01, 0.1, 1, 10, 100]
for n in numbers:
    m = svm_train(trainy2, dtrainx, '-t 2 -g 10 -c {} -q'.format(n))
    acc = svm_predict(testy2, dtestx, m, "-q")[1][0]
    print(n, "Eout: ", round(1-acc/100, 7))


In [129]:
#q15
numbers = [0.1, 1, 10, 100, 1000]
for n in numbers:
    m = svm_train(trainy2, dtrainx, '-t 2 -g {} -c 0.1 -q'.format(n))
    acc = svm_predict(testy2, dtestx, m, "-q")[1][0]
    print(n, "Eout: ", round(1-acc/100, 7))

In [130]:
#q16
count_best=[0]*5
numbers = [0, 1, 2, 3, 4]
for i in range(1000):
    trainx3, valx, trainy3, valy = train_test_split(dtrainx, trainy2, test_size=200, random_state=i+1234)
    bestacc = 0
    bestnum = 0
    for n in numbers:
        m = svm_train(trainy3, trainx3, '-t 2 -g {} -c 0.1 -q'.format(10**(n-1)))
        acc = svm_predict(valy, valx, m, "-q")[1][0]
        if acc > bestacc:
            bestacc = acc
            bestnum = n
    count_best[bestnum]+=1
    print(i, ":", count_best)
print(count_best)